## Reporting: wrangle_report

For the purpose of this project which is basically wrangling and analysing data from Twitter user, WeRateDogs, the data to be used were gathered from different sources. The first data (twitter_archive_enhanced.csv) was downloaded manually and uploaded to the jupyter notebook workspace. The second dataset (image_predictions.tsv) which is a flat file was downloaded and saved programmatically from a provided url. To do this, I used the request library to assess the content of the provided url and wrote it's content into the created 'image_predictions.tsv' file. The third dataset (twitter_api_data.csv) was scraped from twitter by querying its API. The tweets of each ids present in the twitter_archive_enhanced.csv data was collected in an extended mode via api.get_status() of the tweepy library and written into a 'tweet_json.txt' file, each tweets on a new line. The content of this file was then read line by line, and tweet_id, favorite_count and retweet_count extracted from it. The extracted data was converted to a dataframe and stored as 'twitter_api_data.csv'.
Having gathered the required/needed dataset, they were individually assessed both visually and pragmatically (.info(), .duplicated(), .head(), etc) for quality and tidiness issues. The issues detected were documented so they can be fixed. These are;
### Quality issues
1. twitter_archive: in_reply_to_status_id and in_reply_to_user_id columns have 78 values. These are not original ratings. 
This was fixed by removing rows where these columns do not have null values. 

2. twitter_archive: 181 values in the retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp. These are not original ratings.
This was fixed by also removing rows where these columns weren't null. 

3. twitter_archive: in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp columns are inconsequential.
These columns hold records of retweeted tweets and replies to tweets. They aren't tweets made by WeRateDogs account, and since these columns contain null values after eradicating rows where there aren't null values, they are of no use. Hence, dropping them is ideal. 

4. tweet_id in all datasets is of int datatype instead of object. 
The tweet_ids are more of string datatype than numbers. They are means of identification for each tweets made and won't be used for any form of calculations. This was fixed by changing the datatype from 'int' to 'object' or 'str' in each of the dataset. 

5. timestamp is in object datatype instead of datetime. 
This was resolved by using the pd.to_datetime() to change the timestamp datatype from string to datetime. This made it easier to do time-based analysis on the dataset.

6. twitter_archive: Ratings with decimal values incorrectly extracted.
There are ratings that are decimal values. In the enhanced dataset, figures before the decimal points were excluded making the ratings incorrect (75 was extracted in place of 9.75). This was corrected by using a regex that matched and included decimal values and extracted figures both before and after the decimal point from the text column. Then, the values were converted to a float type.

7. twitter_archive: rating_numerator contains ratings that aren't valid (in the actual tweet, these aren't the real ratings, and some have no ratings). It also contains aggregate ratings. These are ratings of pack of dogs.
Some ratings were not correctly extracted. this was detected from their denominators not being 10. Also, some ratings were aggregates. This set of ratings are ratings of two or more dogs given an aggregate rating (this is evident from their denominators being a multiple of 10, the rating scale, and from the text column indicating more than a dog). The invalid raings were corrected manually by replacing the extracted ratings with the correct ones using .loc[] method. The aggregates were corected by dividing the overall ratings by the number of dogs rated (denominator divided by 10). I then dropped rows where there were no ratings because I wanted only rows with ratings.

8. twitter_archive: some values of rating_denominator not 10.
There were rows where rating_denominator was still 10 after correcting their corresponding numerator. This issue was rectified by using the .loc[] method to replace those values with 10.


### Tidiness issues
1. twitter_archive: floofer, doggo, pupper and puppo should be in a column. These are values presented as variables.
These four columns are supposed to be just a single column with the four titles as its values. To achieve this, the melt() method should be used. But there are many rows with all four values as none which means the dog type is unknown. Using the melt() function in this case would result in loss of a large % of the data when the 'none' rows are eventually removed. To maneuver this, I concatenated values in the four columns, replaced 'none' with ' ' and then stripped the resulting values off excess whitespaces. I then replaced where the values are nothing('') with 'none' so that this dogs maintain their non-existent dog-type value.
.value_counts() was used to confirm the list of values in the newly created column. I discovered that there were values with a combination of two dog stages. I did a look-up on these sub-dataframe and discovered some were incorrectly extracted while others were multiple dogs with different dog stage. I corrected the invalid ones by replacing theme with the correct dog_stage.

2. The 3 datsets should be in just a single dataset not separated since they all have a common column (tweet_id). 
All 3 datasets are related and have a common column which is the tweet_id. Thus, they should be in a single sheet. This is achieved by using the .merge() method to join the 3dataframes on the tweet_id. 


After cleaning and fixing all documented data issues, a single dataframe which is a combination of the earlier three dataframes with filtered columns only was arrived at and stored in a new file named 'twitter_archive_master.csv'.